In [1]:
import numpy as np
import pandas as pd


In [2]:
# Simulated stock levels (quantity per SKU)
current_stock = {
    "SKU_1": 120,
    "SKU_2": 90,
    "SKU_3": 45,
}

In [3]:


# Forecasted demand (from your model)
# You will connect real predictions in Module 4
forecast_demand = {
    "SKU_1": np.array([20, 25, 22, 18]),
    "SKU_2": np.array([15, 18, 21, 20]),
    "SKU_3": np.array([10, 12, 15, 16]),
}

In [4]:



# Cost settings
holding_cost_per_unit = 0.5
stockout_cost_per_unit = 4.0
lead_time_hours = 2


In [5]:
def reorder_qty_calc(sku, stock, forecast):
    demand_during_lead = forecast[:lead_time_hours].sum()

    # If running out of stock soon, reorder
    if stock < demand_during_lead:
        reorder_qty = (demand_during_lead - stock) + 20  # safety buffer
    else:
        reorder_qty = 0
    
    return max(reorder_qty, 0)

In [6]:


reorder_plan = {}
for sku, stock in current_stock.items():
    reorder_plan[sku] = reorder_qty_calc(sku, stock, forecast_demand[sku])

reorder_plan


{'SKU_1': 0, 'SKU_2': 0, 'SKU_3': 0}

In [8]:
def simulate_cost(stock, forecast):
    shortages = np.maximum(forecast - stock, 0)
    stockouts_cost = shortages.sum() * stockout_cost_per_unit

    remaining = stock - forecast.sum()
    holding_cost = max(remaining, 0) * holding_cost_per_unit
    return stockouts_cost + holding_cost

cost_no_restock = {
    sku: simulate_cost(stock, forecast_demand[sku])
    for sku, stock in current_stock.items()
}

# apply reorder plan
cost_with_restock = {}
for sku in current_stock:
    updated_stock = current_stock[sku] + reorder_plan[sku]
    cost_with_restock[sku] = simulate_cost(updated_stock, forecast_demand[sku])


In [11]:
cost_comparison = pd.DataFrame([
    {
        "SKU": sku,
        "No_Restock": cost_no_restock[sku],
        "With_Restock": cost_with_restock[sku],
        "Savings": cost_no_restock[sku] - cost_with_restock[sku]
    }
    for sku in cost_no_restock.keys()
])

cost_comparison = cost_comparison.set_index("SKU")
cost_comparison


,No_Restock,With_Restock,Savings
SKU,,,
SKU_1,17.5,17.5,0.0
SKU_2,8.0,8.0,0.0
SKU_3,0.0,0.0,0.0
